https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset

In [ ]:
#imports
import pandas as pd
import numpy as np

In [ ]:
#Load fake news data
df_fake = pd.read_csv('Fake.csv')
df_fake['label'] = 0
df_fake.head()

In [ ]:
#Load real news data
df_true = pd.read_csv('True.csv')
df_true['label'] = 1
df_true.head()

In [ ]:
#merge & shuffle data
df = pd.concat([df_fake, df_true]).sample(frac=1).reset_index(drop=True)
df.head(50)